In [497]:
import pyshacl
from rdflib import OWL, RDF, RDFS, SH, Graph, Namespace

# Testing on SPARQL info addition

In [686]:
file = 'SPARQL-test.ttl'
file = 'construct_typing.ttl'

g = Graph ()
g.parse(file)
sg = Graph()
sg.parse(file)

# graph before validation
og = Graph()
pyshacl.rdfutil.clone.clone_graph(g, og)

<Graph identifier=N6cc7aa6208634694b18f4f9cdca6dd95 (<class 'rdflib.graph.Graph'>)>

In [687]:
valid, report_graph, report_text = pyshacl.validate(
    data_graph=g,
    shacl_graph=sg,
    advanced=True,
    js=True,
    allow_warnings=False,
    inplace=True,
    iterate_rules=True,
)

In [688]:
# removing the original graph information from the validated/inferenced graph
added_triples = g - og

In [689]:
added_triples.print()


<http://example.org/testing#chaining2> a <http://example.org/testing#chaining_nodeshape2> .

<http://example.org/testing#chaining3> a <http://example.org/testing#chaining_nodeshape3> .




In [691]:
namespace_map = {}
for prefix, uriref in report_graph.namespaces():
    namespace_map[prefix] = Namespace(uriref)
    # Need OR for result Message
qs = """
    SELECT ?focusNode ?resultMessage ?resultSeverity ?resultPath
    WHERE {
        BIND(sh:Violation AS ?resultSeverity) .
        ?report rdf:type sh:ValidationReport .
        ?report sh:result ?result .
        ?result sh:focusNode ?focusNode .
        OPTIONAL {?result sh:resultMessage ?resultMessage} .
        ?result sh:resultPath ?resultPath .
        ?result sh:resultSeverity ?resultSeverity .
        }
    """

# pretty colors
color_map = {SH.Violation: 33, SH.Info: 34, SH.Warning: 35}

# query
results = sorted(report_graph.query(qs, initNs=namespace_map))

prev = None
for focusNode, resultMessage,resultSeverity, resultPath in results:
    if focusNode != prev:
        print(focusNode)
        prev = focusNode
    color = color_map[resultSeverity]
    print(f"\x1b[{color}m    {resultMessage}\x1b[0m   {resultPath}\x1b[0m")

http://example.org/testing#chaining2
    Value does not conform to Shape test:chaining_nodeshape3   http://example.org/testing#chain_prop
http://example.org/testing#chaining3
    bottom level propshape   http://example.org/testing#chain_prop


In [692]:
print(report_text)

Validation Report
Conforms: False
Results (2):
Constraint Violation in NodeConstraintComponent (http://www.w3.org/ns/shacl#NodeConstraintComponent):
	Severity: sh:Violation
	Source Shape: test:cps2
	Focus Node: test:chaining2
	Value Node: test:chaining3
	Result Path: test:chain_prop
	Message: Value does not conform to Shape test:chaining_nodeshape3
Constraint Violation in HasValueConstraintComponent (http://www.w3.org/ns/shacl#HasValueConstraintComponent):
	Severity: sh:Violation
	Source Shape: test:cps3
	Focus Node: test:chaining3
	Result Path: test:chain_prop
	Message: bottom level propshape



# recursive targetting

## queries

In [835]:
query_prop_all = """
PREFIX sh: <http://www.w3.org/ns/shacl#> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

CONSTRUCT {
    $this sh:rule _:rule .
    _:rule a sh:TripleRule ;
        sh:object ?object ;
        sh:predicate rdf:type ;
        sh:subject _:subnode ;
    .
    _:subnode sh:path ?path ;
    .
 }
WHERE {
$this a sh:NodeShape ;
    (sh:or|sh:and)/rdf:rest*/rdf:first | sh:property ?propshape .
?propshape sh:path ?path;
    sh:node ?object .
}
"""
query_node_all = """
PREFIX sh: <http://www.w3.org/ns/shacl#> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

CONSTRUCT {
    $this sh:rule _:rule .
    _:rule a sh:TripleRule ;
        sh:object ?object ;
        sh:predicate rdf:type ;
        sh:subject sh:this ;
    .
 }
WHERE {
    $this a sh:NodeShape ;
    (sh:or|sh:and)/rdf:rest*/rdf:first | sh:node ?object.
}
"""

## Running Rules on various different patterns

In [836]:
def run_validation(file):
    g = Graph ()
    g.parse(file)

    # graph before validation
    og = Graph()
    pyshacl.rdfutil.clone.clone_graph(g, og)

    g = g.query(query_prop_all).graph + g
    g = g.query(query_node_all).graph + g

    #shapes graph same as datagraph here
    sg = Graph()
    pyshacl.rdfutil.clone.clone_graph(g, sg)

    valid, report_graph, report_text = pyshacl.validate(
        data_graph=g,
        shacl_graph=sg,
        advanced=True,
        js=True,
        allow_warnings=False,
        inplace=True,
        iterate_rules=True, # If you iterate rules, this SPARQL construct for chain_rule won't work
    )
    # removing the original graph information from the validated/inferenced graph
    # added_triples = g - og
    # added_triples.print()
    namespace_map = {}
    for prefix, uriref in report_graph.namespaces():
        namespace_map[prefix] = Namespace(uriref)
        # Need OR for result Message
    qs = """
        SELECT ?focusNode ?resultMessage ?resultSeverity ?resultPath
        WHERE {
        #    BIND(sh:Violation AS ?resultSeverity) .
            ?report rdf:type sh:ValidationReport .
            ?report sh:result ?result .
            ?result sh:focusNode ?focusNode .
            OPTIONAL {?result sh:resultMessage ?resultMessage} .
            ?result sh:resultPath ?resultPath .
            ?result sh:resultSeverity ?resultSeverity .
            }
        """

    # pretty colors
    color_map = {SH.Violation: 33, SH.Info: 34, SH.Warning: 35}

    # query
    results = sorted(report_graph.query(qs, initNs=namespace_map))

    prev = None
    for focusNode, resultMessage,resultSeverity, resultPath in results:
        if focusNode != prev:
            print(focusNode)
            prev = focusNode
        color = color_map[resultSeverity]
        print(f"\x1b[{color}m    {resultMessage}\x1b[0m   {resultPath}\x1b[0m")
    # recursive targetting

In [837]:
files = ['chaining_shapes.ttl','chaining_and_or.ttl']
for file in files:
    print(file)
    run_validation(file)

chaining_shapes.ttl
http://example.org/testing#chaining
    Value does not conform to Shape test:chaining_nodeshape2   http://example.org/testing#chain_prop
http://example.org/testing#chaining2
    Value does not conform to Shape test:chaining_nodeshape3   http://example.org/testing#chain_prop
http://example.org/testing#chaining3
    bottom level propshape   http://example.org/testing#chain_prop
chaining_and_or.ttl
http://example.org/testing#fail_and_succeed_or
    bottom level constraint, directly failed, message should report   http://example.org/testing#test_prop
http://example.org/testing#fail_or_succeed_and
    bottom level constraint, directly failed, message should report   http://example.org/testing#test_prop
    bottom level constraint, directly failed, message should report   http://example.org/testing#test_prop


## Dev

In [ ]:
query_terminal_prop = """
PREFIX sh: <http://www.w3.org/ns/shacl#> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

CONSTRUCT {
    $this sh:rule _:rule .
    _:rule a sh:TripleRule ;
        sh:object ?propshape ;
        sh:predicate sh:property ;
        sh:subject sh:this ;
    .
    $this sh:rule _:rule2 .
    _:rule2 a sh:TripleRule ;
        sh:object sh:NodeShape ;
        sh:predicate rdf:type ;
        sh:subject sh:this ;
    .

 }
WHERE {
$this a sh:NodeShape ;
    (sh:or|sh:and)/rdf:rest*/rdf:first | sh:property ?propshape .
?propshape a sh:PropertyShape .
}
"""

In [824]:
file = 'SPARQL-test.ttl'
file = 'construct_typing.ttl'
file = 'chaining_shapes.ttl'
file = 'chaining_and_or.ttl'
file = 'sh-and-or-test.ttl'

g = Graph ()
g.parse(file)

# graph before validation
og = Graph()
pyshacl.rdfutil.clone.clone_graph(g, og)

g = g.query(query_prop_all).graph + g
g = g.query(query_node_all).graph + g
g = g.query(query_terminal_prop).graph + g

#shapes graph same as datagraph here
sg = Graph()
pyshacl.rdfutil.clone.clone_graph(g, sg)
# added_triples = g - og
# added_triples.print()

<Graph identifier=N34b6080ad57141028cb8c93303fad3f8 (<class 'rdflib.graph.Graph'>)>

In [828]:
valid, report_graph, report_text = pyshacl.validate(
    data_graph=g,
    shacl_graph=sg,
    advanced=True,
    js=True,
    allow_warnings=False,
    inplace=True,
    iterate_rules=True, # If you iterate rules, this SPARQL construct for chain_rule won't work
)
# removing the original graph information from the validated/inferenced graph
# added_triples = g - og
# added_triples.print()
namespace_map = {}
for prefix, uriref in report_graph.namespaces():
    namespace_map[prefix] = Namespace(uriref)
    # Need OR for result Message
qs = """
    SELECT ?focusNode ?resultMessage ?resultSeverity ?resultPath
    WHERE {
    #    BIND(sh:Violation AS ?resultSeverity) .
        ?report rdf:type sh:ValidationReport .
        ?report sh:result ?result .
        ?result sh:focusNode ?focusNode .
        OPTIONAL {?result sh:resultMessage ?resultMessage} .
        ?result sh:resultPath ?resultPath .
        ?result sh:resultSeverity ?resultSeverity .
        }
    """

# pretty colors
color_map = {SH.Violation: 33, SH.Info: 34, SH.Warning: 35}

# query
results = sorted(report_graph.query(qs, initNs=namespace_map))

prev = None
for focusNode, resultMessage,resultSeverity, resultPath in results:
    if focusNode != prev:
        print(focusNode)
        prev = focusNode
    color = color_map[resultSeverity]
    print(f"\x1b[{color}m    {resultMessage}\x1b[0m   {resultPath}\x1b[0m")
# recursive targetting

@prefix ns1: <http://www.w3.org/ns/shacl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .

<http://example.org/testing#constraint> ns1:rule [ a ns1:TripleRule ;
            ns1:object <http://example.org/testing#or_prop> ;
            ns1:predicate rdf:type ;
            ns1:subject ns1:this ],
        [ a ns1:TripleRule ;
            ns1:object <http://example.org/testing#and_prop> ;
            ns1:predicate rdf:type ;
            ns1:subject ns1:this ] .

<http://example.org/testing#fail_and_succeed_or> a <http://example.org/testing#0_or_less>,
        <http://example.org/testing#1_or_less>,
        <http://example.org/testing#1_or_more>,
        <http://example.org/testing#2_or_more>,
        <http://example.org/testing#and_prop>,
        <http://example.org/testing#or_prop>,
        ns1:NodeShape ;
    ns1:property <http://example.org/testing#0_or_less>,
        <http://example.org/testing#1_or_less>,
        <http://example.org/testing#1_or_more>,
        <http://

In [829]:
print(report_text)

Validation Report
Conforms: False
Results (4):
Constraint Violation in OrConstraintComponent (http://www.w3.org/ns/shacl#OrConstraintComponent):
	Severity: sh:Violation
	Source Shape: test:or_prop
	Focus Node: test:fail_or_succeed_and
	Value Node: test:fail_or_succeed_and
	Message: 2nd level and constraint
Constraint Violation in AndConstraintComponent (http://www.w3.org/ns/shacl#AndConstraintComponent):
	Severity: sh:Violation
	Source Shape: test:and_prop
	Focus Node: test:fail_and_succeed_or
	Value Node: test:fail_and_succeed_or
	Message: 2nd level and constraint
Constraint Violation in NodeConstraintComponent (http://www.w3.org/ns/shacl#NodeConstraintComponent):
	Severity: sh:Violation
	Source Shape: test:constraint
	Focus Node: test:fail_and_succeed_or
	Value Node: test:fail_and_succeed_or
	Message: top level constraint
Constraint Violation in NodeConstraintComponent (http://www.w3.org/ns/shacl#NodeConstraintComponent):
	Severity: sh:Violation
	Source Shape: test:constraint
	Focus 

In [830]:


#queryold queries
query_node = """
PREFIX sh: <http://www.w3.org/ns/shacl#> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

CONSTRUCT {
    $this sh:rule _:rule .
    _:rule a sh:TripleRule ;
        sh:object ?object ;
        sh:predicate rdf:type ;
        sh:subject sh:this ;
    .
 }
WHERE {
$this a sh:NodeShape ;
    sh:node ?object .
}
"""
query_and = """
PREFIX sh: <http://www.w3.org/ns/shacl#> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

CONSTRUCT {
    $this sh:rule _:rule .
    _:rule a sh:TripleRule ;
        sh:object ?object ;
        sh:predicate rdf:type ;
        sh:subject sh:this ;
    .
 }
WHERE {
    $this a sh:NodeShape ;
    sh:and/rdf:rest*/rdf:first ?object.
}
"""
query_or = """
PREFIX sh: <http://www.w3.org/ns/shacl#> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

CONSTRUCT {
    $this sh:rule _:rule .
    _:rule a sh:TripleRule ;
        sh:object ?object ;
        sh:predicate rdf:type ;
        sh:subject sh:this ;
    .
 }
WHERE {
    $this a sh:NodeShape ;
    sh:or/rdf:rest*/rdf:first ?object.
}
"""
query_node_all = """
PREFIX sh: <http://www.w3.org/ns/shacl#> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

CONSTRUCT {
    $this sh:rule _:rule .
    _:rule a sh:TripleRule ;
        sh:object ?object ;
        sh:predicate rdf:type ;
        sh:subject sh:this ;
    .
 }
WHERE {
    $this a sh:NodeShape ;
    sh:or/rdf:rest*/rdf:first | sh:and/rdf:rest*/rdf:first | sh:node ?object.
}
"""